In [2]:
!pip install opencv-python numpy


In [12]:
import cv2
import numpy as np
import os
import time
from collections import deque

# Save directly to Downloads folder
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
print(f"📁 Screenshots will save to: {downloads_path}")

# Initialize camera
cap = cv2.VideoCapture(0)
stress_history = deque(maxlen=90)  # 30 seconds of data
consecutive_stress = 0
alert_threshold = 4

print("Press 'S' to save screenshot to Downloads, 'Q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        # Create demo frame if camera fails
        frame = np.ones((600, 900, 3), dtype=np.uint8) * 50  # Dark background
        cv2.putText(frame, "CAMERA NOT AVAILABLE - DEMO MODE", (250, 300), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    else:
        frame = cv2.flip(frame, 1)  # Mirror effect
    
    h, w = frame.shape[:2]
    
    # Make the frame larger if needed
    if h < 600 or w < 900:
        frame = cv2.resize(frame, (900, 600))
        h, w = 600, 900
    
    # Simulate face detection and emotions
    face_x, face_y = 50, 100
    face_w, face_h = 300, 350
    
    # Cycle between stress and normal states
    current_time = time.time()
    if int(current_time) % 8 < 4:  # Stress state for 4 seconds
        emotions = ["NEUTRAL", "SAD"]
        stress_level = 0.85
    else:  # Normal state for 4 seconds
        emotions = ["HAPPY"]
        stress_level = 0.25
    
    # Update stress history for timeline
    stress_history.append(stress_level)
    
    # Update alert counter
    if stress_level > 0.6:
        consecutive_stress += 1
    else:
        consecutive_stress = 0
    
    # ==================== LIVE CAMERA VIEW ====================
    # Draw face bounding box
    cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 255, 0), 3)
    
    # Add emotion labels above face box
    for i, emotion in enumerate(emotions):
        text_y = face_y - 30 - (i * 35)
        # Add background for text
        text_size = cv2.getTextSize(emotion, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
        cv2.rectangle(frame, (face_x-5, text_y-text_size[1]-5), 
                     (face_x + text_size[0] + 5, text_y+5), (0, 0, 0), -1)
        cv2.putText(frame, emotion, (face_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    # Add "Face Detected" label below the face box
    cv2.putText(frame, "Face Detected", (face_x + 80, face_y + face_h + 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # ==================== STRESS-CONFIDENCE TIMELINE ====================
    timeline_x, timeline_y = 400, 100
    timeline_w, timeline_h = 450, 150
    
    # Draw timeline background
    cv2.rectangle(frame, (timeline_x, timeline_y), 
                 (timeline_x + timeline_w, timeline_y + timeline_h), 
                 (30, 30, 30), -1)
    cv2.rectangle(frame, (timeline_x, timeline_y), 
                 (timeline_x + timeline_w, timeline_y + timeline_h), 
                 (255, 255, 255), 2)
    
    # Draw grid lines and labels
    for i in range(0, 11):
        y_pos = timeline_y + int(i * 0.1 * timeline_h)
        if i % 2 == 0:
            cv2.line(frame, (timeline_x, y_pos), (timeline_x + timeline_w, y_pos), 
                     (100, 100, 100), 1)
            # Add Y-axis labels
            label = f"{1.0 - i*0.1:.1f}"
            cv2.putText(frame, label, (timeline_x - 25, y_pos + 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
    
    # Draw stress line if we have data
    if len(stress_history) > 1:
        points = []
        for i, stress_val in enumerate(stress_history):
            x = timeline_x + int((i / len(stress_history)) * timeline_w)
            y = timeline_y + timeline_h - int(stress_val * timeline_h)
            points.append((x, y))
        
        # Draw the stress line
        for i in range(1, len(points)):
            color = (0, 0, 255) if stress_history[i] > 0.6 else (0, 255, 0)
            cv2.line(frame, points[i-1], points[i], color, 3)
    
    # Draw threshold line
    threshold_y = timeline_y + timeline_h - int(0.6 * timeline_h)
    cv2.line(frame, (timeline_x, threshold_y), (timeline_x + timeline_w, threshold_y), 
             (255, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Threshold 0.6", (timeline_x + 10, threshold_y - 5), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
    
    # Add timeline title
    cv2.putText(frame, "Stress-Confidence Timeline (30-second window)", 
               (timeline_x, timeline_y - 10), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # ==================== ALERT SYSTEM ====================
    alert_x, alert_y = 400, 280
    alert_w, alert_h = 450, 100
    
    # Check if alert should be activated
    alert_active = consecutive_stress >= alert_threshold
    
    if alert_active:
        # Red alert - threshold reached
        cv2.rectangle(frame, (alert_x, alert_y), 
                     (alert_x + alert_w, alert_y + alert_h), 
                     (0, 0, 200), -1)
        cv2.rectangle(frame, (alert_x, alert_y), 
                     (alert_x + alert_w, alert_y + alert_h), 
                     (0, 0, 255), 3)
        
        alert_text = "TAKE BREAK"
        text_size = cv2.getTextSize(alert_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0]
        text_x = alert_x + (alert_w - text_size[0]) // 2
        text_y = alert_y + (alert_h + text_size[1]) // 2
        
        cv2.putText(frame, alert_text, (text_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
        
        # Add alert info
        info_text = f"ALERT! {consecutive_stress} consecutive stress frames detected"
        cv2.putText(frame, info_text, (alert_x + 20, alert_y + alert_h + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        # Normal state
        cv2.rectangle(frame, (alert_x, alert_y), 
                     (alert_x + alert_w, alert_y + alert_h), 
                     (50, 50, 50), -1)
        cv2.rectangle(frame, (alert_x, alert_y), 
                     (alert_x + alert_w, alert_y + alert_h), 
                     (150, 150, 150), 2)
        
        status_text = f"Stress Frames: {consecutive_stress}/{alert_threshold}"
        text_size = cv2.getTextSize(status_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
        text_x = alert_x + (alert_w - text_size[0]) // 2
        text_y = alert_y + (alert_h + text_size[1]) // 2
        
        cv2.putText(frame, status_text, (text_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        cv2.putText(frame, "Monitoring...", (alert_x + alert_w//2 - 50, alert_y + alert_h - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    
    # ==================== STATUS INFO ====================
    info_y = 500
    cv2.putText(frame, f"Current Emotions: {', '.join(emotions)}", (50, info_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(frame, f"Stress Level: {stress_level:.3f}", (50, info_y + 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255) if stress_level > 0.6 else (0, 255, 0), 2)
    cv2.putText(frame, f"Status: {'HIGH STRESS' if stress_level > 0.6 else 'NORMAL'}", (50, info_y + 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255) if stress_level > 0.6 else (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Stress Detection System', frame)
    
    # Handle key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == ord('Q'):
        break
    elif key == ord('s') or key == ord('S'):
        filename = f"stress_detection_{int(time.time())}.png"
        full_path = os.path.join(downloads_path, filename)
        cv2.imwrite(full_path, frame)
        print(f"✅ Screenshot saved to Downloads: {filename}")
        print(f"📍 Location: {downloads_path}")

cap.release()
cv2.destroyAllWindows()

📁 Screenshots will save to: C:\Users\krish\Downloads
Press 'S' to save screenshot to Downloads, 'Q' to quit
✅ Screenshot saved to Downloads: stress_detection_1762408290.png
📍 Location: C:\Users\krish\Downloads
✅ Screenshot saved to Downloads: stress_detection_1762408299.png
📍 Location: C:\Users\krish\Downloads
